In [6]:
import torch
import torch.nn as nn


class block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(block, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride, 1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.bn2(self.conv2(self.relu(self.bn1(self.conv1(x))))) + self.shortcut(
            x
        )

In [7]:
class resnet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(resnet, self).__init__()

        self.in_channels = 64
        self.conv1 = nn.Conv2d(in_channels, 64, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(64, 2, 1)
        self.layer2 = self._make_layer(128, 2, 2)
        self.layer3 = self._make_layer(256, 2, 2)
        self.layer4 = self._make_layer(512, 2, 2)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.fc(
            torch.flatten(
                torch._adaptive_avg_pool2d(
                    self.layer4(
                        self.layer3(
                            self.layer2(
                                self.layer1(torch.relu(self.bn1(self.conv1(x))))
                            )
                        )
                    ),
                    1,
                    1,
                )
            )
        )

    def _make_layer(self, out_channels, blocks, stride):
        layers = [block(self.in_channels, out_channels, stride)]
        self.in_channels = out_channels

        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)